In [1]:
import os
import dotenv

import pymongo
import pandas as pd

In [3]:
def connect_to_db_Wetter_Durchschnitt():
    # Load environment variables from .env file
    dotenv.load_dotenv()
    
    # Get MongoDB-URI
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["MDM-Python-MeinProjekt"]

    if "Wetter_Durchschnitt" in db.list_collection_names():
        return db["Wetter_Durchschnitt"]
    else:
        collection = db["Wetter_Durchschnitt"]
        collection.create_index([
            ("date", pymongo.ASCENDING),
        ], unique=True)
        return collection

In [68]:
def extract_average_weather():

    collection = connect_to_db_Wetter_Durchschnitt()
    
    pipeline = [
      {
        '$project': {
          '_id': False,
          'date': "$date",
          'avg_temp': "$avg_temp",
          'min_temp': "$min_temp",
          'max_temp': "$max_temp",
          'rain': "$rain",
          'wind_speed': "$wind_speed",
          'clouds': "clouds",
        },
      },
    ]
    
    
    results = []
    for x in collection.aggregate(pipeline):
        results.append(x)
    
    df = pd.DataFrame(results)
    df = df.set_index("date")
    df = df.set_index(pd.to_datetime(df.index).tz_localize("UTC"))
    df = df.sort_index()

    return df

In [62]:
df = extract_average_weather()

In [63]:
df

,avg_temp,min_temp,max_temp,rain,wind_speed,clouds
date,,,,,,
2022-05-24 00:00:00+00:00,10.493056,3.4,19.9,0.264525,1.396862,clouds
2022-05-25 00:00:00+00:00,11.242477,1.7,23.0,0.081238,1.178819,clouds
2022-05-25 00:00:00+00:00,11.242477,1.7,23.0,0.081238,1.178819,clouds
2022-05-26 00:00:00+00:00,13.049653,0.3,26.4,0.019502,1.432935,clouds
2022-05-26 00:00:00+00:00,13.049653,0.3,26.4,0.019502,1.432935,clouds
...,...,...,...,...,...,...
2024-03-08 00:00:00+00:00,0.025000,-12.3,11.0,0.020938,2.567483,clouds
2024-03-09 00:00:00+00:00,2.076389,-10.3,13.7,0.069630,2.857928,clouds
2024-03-09 00:00:00+00:00,2.076389,-10.3,13.7,0.069630,2.857928,clouds
